In [1]:
import pandas as pd
from sqlalchemy import create_engine

import plotly
import plotly.graph_objs as go

import json

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import math

In [2]:
database_filepath = 'DisasterResponse.db'

In [3]:
# Create the engine
engine = create_engine(f"sqlite:///{database_filepath}")
# Load the data in a pandas Dataframe
df = pd.read_sql_table("disaster_messages", engine)

In [4]:
def get_graph_json(graph):
    return json.dumps(graph, cls=plotly.utils.PlotlyJSONEncoder)

# Example already in script

In [5]:
# genre_counts = df.groupby('genre').count()['message']
# genre_names = list(genre_counts.index)

# genre_counts_graph = {
#     'data': [
#         go.Bar(
#             x=genre_names,
#             y=genre_counts
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Distribution of Message Genres'
#         },
#         'yaxis': {
#             'title': {
#                 'text': "Count"
#             }
#         },
#         'xaxis': {
#             'title': {
#                 'text': "Genre"
#             }
#         }
#     }
# }

# plotly.io.from_json( get_graph_json(genre_counts_graph) )

# Counts per Message Category

In [6]:
# msg_categories = df.columns[4:]

# msg_category_cols = df.columns[4:]
# cat_count = df[msg_category_cols].sum().sort_values(ascending=False).head(10)
# msg_category_display = [ cn.replace("_", " ").capitalize() for cn in cat_count.index ]

# cat_counts_graph = {
#     'data': [
#         go.Bar(
#             x=msg_category_display,
#             y=cat_count
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Count of Messages by Category'
#         },
#         'yaxis': {
#             'title': {
#                 'text': "Count"
#             }
#         },
#         'xaxis': {
#             'title': {
#                 'text': "Message Category"
#             }
#         }
#     }
# }

# plotly.io.from_json( get_graph_json(cat_counts_graph) )

# Counts of translated message - Bar

In [7]:
# # df['diff_lang'] = df['message'] != df['original']
# df['diff_lang'] = df.apply( lambda row: 'Translated' if row['message'] != row['original'] else "English", axis=1 )
# diff_lang_count = df.groupby('diff_lang').count()['message'].reset_index().rename(columns={'message': 'count'})

# display(diff_lang_count)

# diff_lang_graph = {
#     'data': [
#         go.Bar(
#             x=diff_lang_count['diff_lang'],
#             y=diff_lang_count['count']
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Count of Messages Not in English'
#         },
#         'yaxis': {
#             'title': {
#                 'text': "Count"
#             }
#         },
#         'xaxis': {
#             'title': {
#                 'text': "Translation"
#             }
#         }
#     }
# }

# plotly.io.from_json( get_graph_json(diff_lang_graph) )

# Count of translated messages - Pie

In [8]:
# df['diff_lang'] = df.apply( lambda row: 'Translated' if row['message'] != row['original'] else "English", axis=1 )
# diff_lang_count = df.groupby('diff_lang').count()['message'].reset_index().rename(columns={'message': 'count'})

# diff_lang_graph = {
#     'data': [
#         go.Pie(
#             labels=diff_lang_count['diff_lang'], values=diff_lang_count['count']
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Count of Messages Not in English'
#         }
#     }
# }

# plotly.io.from_json( get_graph_json(diff_lang_graph) )

# Counts of words per message - Pie chart

In [9]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [10]:
# word_count = df['message'].apply( lambda msg: len(tokenize(msg)) ).rename('word_count').reset_index(drop=True)
# word_count_bins = pd.cut(word_count, bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, float('inf')], labels=['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100+'])
# count_per_bin = word_count_bins.value_counts().sort_index().reset_index().rename(columns={'word_count': 'bins'})
# display( count_per_bin )

# word_count_graph = {
#     'data': [
#         go.Pie(
#             labels=count_per_bin['bins'], values=count_per_bin['count'], sort=False
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Counts of Message Lengths'
#         }
#     }
# }

# plotly.io.from_json( get_graph_json(word_count_graph) )

# Counts of words per message - Histogram

In [11]:
# import plotly.figure_factory as ff

# word_count = df['message'].apply( lambda msg: len(tokenize(msg)) ).rename('word_count').reset_index(drop=True)
# word_count_u100 = word_count[word_count < 100]
# word_count_o100 = word_count[word_count >= 100]

# fig1 = ff.create_distplot([word_count_u100], ['Word Count (under 100 words)'], bin_size=1)
# fig2 = ff.create_distplot([word_count_o100], ['Word Count (over 100 words)'], bin_size=100)
# fig1.show()
# fig2.show()

# Message Categories by Count by Word Count

In [ ]:
# group the dataframe by word_count bin and category and get the count for each group.
# x axis can be the word_count bin, y axis can be the count, and the size of the bubble can be the category count

cat_wordcount_counts = df.copy()

bins = [0, 10, 20, 30, 40, 50, float('inf')]
labels=['0-9', '10-19', '20-29', '30-39', '40-49', '50+']
word_count = cat_wordcount_counts['message'].apply( lambda msg: len(tokenize(msg)) ).rename('word_count').reset_index(drop=True)
cat_wordcount_counts['word_count_bins'] = pd.cut( word_count, bins=bins, labels=labels )

msg_category_cols = df.columns[4:]
cat_count = cat_wordcount_counts[msg_category_cols].sum().sort_values(ascending=False)
msg_category_display = [ cn.replace("_", " ").capitalize() for cn in cat_count.index ]

cat_wordcount_counts = cat_wordcount_counts[['word_count_bins', *msg_category_cols]].groupby(['word_count_bins']).sum().reset_index()

count_by_category_and_bin = pd.melt( cat_wordcount_counts, id_vars=['word_count_bins'], value_vars=msg_category_cols, var_name='category', value_name='count' )
count_by_category_and_bin['marker_size'] = count_by_category_and_bin['word_count_bins'].apply( lambda x: int(x.split('-')[0]) + 10 if x != '50+' else 60 )

total_counts = count_by_category_and_bin.groupby('category')['count'].sum().reset_index().rename(columns={'count': 'total_count'})

count_by_category_and_bin = count_by_category_and_bin.merge(total_counts, on='category')
count_by_category_and_bin['percentage'] = count_by_category_and_bin['count'] / count_by_category_and_bin['total_count'] * 100
count_by_category_and_bin['percentage'] = count_by_category_and_bin['percentage'].apply( lambda x: round(x, 2) )

count_by_category_and_bin.sort_values(by=['category', 'word_count_bins'], ascending=[True, False], inplace=True)

fig = go.Figure()

## Good at showing the count of messages by category and word count, but the categories are the same color
# fig.add_trace(
#     go.Scatter(
#         x=count_by_category_and_bin['category'].apply( lambda x: x.replace("_", " ").capitalize() ),
#         y=count_by_category_and_bin['word_count_bins'],
#         name='Count',
#         mode='markers',
#         marker=dict(
#             size=count_by_category_and_bin['marker_size']
#         )
#     )
# )

## Good for showing the categories as their own color
# for category in msg_category_cols:
#     category_data = count_by_category_and_bin[count_by_category_and_bin['category'] == category]
#     fig.add_trace(
#         go.Scatter(
#             x=category_data['category'].apply( lambda x: x.replace("_", " ").capitalize() ),
#             y=category_data['count'],
#             name=category,
#             text=category_data['word_count_bins'],
#             mode='markers',
#             marker=dict(
#                 size=category_data['marker_size']
#             )
#         )
#     )

# fig.update_layout(
#     title='Count of Messages by Category and Word Count',
#     xaxis=dict(
#         title='Category'
#     ),
#     yaxis=dict(
#         title='Message Count',
#         type='log'
#     )
# )

## shows the bar chart, but has separate bar for each category
# for category in msg_category_cols:
#     category_data = count_by_category_and_bin[count_by_category_and_bin['category'] == category]
#     fig.add_trace(
#         go.Bar(
#             x=category_data['word_count_bins'],
#             y=category_data['count'],
#             name=category,
#             text=category_data['count'],
#             textposition='auto'
#         )
#     )

# ## creating a stacked barchart to show percentages per category - This is the final form, need to convert to json
# import plotly.express as px
# fig = px.bar(count_by_category_and_bin, x='category', y='percentage', color='word_count_bins', barmode='stack')

# fig.show()

#https://plotly.com/javascript/reference/index/
percentage_per_bin_graph = {
    'data': [
        go.Bar(
            x=count_by_category_and_bin['category'],
            y=count_by_category_and_bin['percentage'],
            color=count_by_category_and_bin['word_count_bins'],
            text=count_by_category_and_bin['percentage']
        )
    ],

    'layout': {
        'title': {
            'text': 'Percentage of Messages by Category and Word Count'
        },
        'yaxis': {
            'title': {
                'text': "Message Category"
            }
        },
        'xaxis': {
            'title': {
                'text': "Percentage of total"
            }
        }
    }
}

plotly.io.from_json( get_graph_json(percentage_per_bin_graph) )

/var/folders/_c/rhw8y9f15l75mmrds0nnkn340000gp/T/ipykernel_22372/2006729108.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



ValueError: Invalid property specified for object of type plotly.graph_objs.Bar: 'color'

Did you mean "zorder"?

    Valid properties:
        alignmentgroup
            Set several traces linked to the same position axis or
            matching axes to the same alignmentgroup. This controls
            whether bars compute their positional range dependently
            or independently.
        base
            Sets where the bar base is drawn (in position axis
            units). In "stack" or "relative" barmode, traces that
            set "base" will be excluded and drawn in "overlay" mode
            instead.
        basesrc
            Sets the source reference on Chart Studio Cloud for
            `base`.
        cliponaxis
            Determines whether the text nodes are clipped about the
            subplot axes. To show the text nodes above axis lines
            and tick labels, make sure to set `xaxis.layer` and
            `yaxis.layer` to *below traces*.
        constra